# Reproducibility Challenge
# Sébastien Pereira et Valentin Noël
# Master DAC
# Apprentissage Statistique

## Introduction

L'objectif de l'article est de fournir un modèle d'apprentissage automatique permettant de générer des dictionnaire d'une langue source vers une langue cible dans un paradigme entièrement non supervisé. 

L'idée principale est de trouver une transformation dans l'espace vectoriel des mots qui aux représentations des mots d'une langue source font correspondre les représentations des mots d'une langue cible. 
Dans cet article nous supposons que nous avons deux espaces vectoriels correspondant à nos deux ensembles de données, source et cible. Ces espaces correspondent aux embbeding monolingue des langue source et cible obtenus de manière indépendante.

L'apprentissage se décompose en deux parties:
- Apprentissage adverse : dans cette partie le but est d'obtenir une première approximation de la transformation de l'espace qui définira notre fonction de mapping de la source vers la cible.
- Analyse procrustéenne :  dans cette partie nous utilisons quelques points particuiers obtenus à l'étape précédente et nous résolvons un problème de Procuste orthogonal, ces étapes sont répétées de manière itérative jusqu'à convergence.

L'inférence s'obtient par la suite grâce à la transformation de l'espace obtenue appliquée aux éléments de la source, puis en utilisant une mesure de similarité CSLS qui permet de résoudre le problème des hubs des KNN (points qui tendent à être les PPV de beaucoup de points en grande dimension -curse of dimensionality).

# Apprentissage adverse

In [58]:
import torch
from torch.nn import Module, Linear, LeakyReLU, LogSoftmax, NLLLoss
from torch.autograd import Variable
import numpy as np

In [27]:
class Generator(Module):
    def __init__(self, size):
        '''
        :param size: size of the embbeding space transformation
        '''
        super(Generator,self).__init__()
        self.transformation = Linear(size, size)
        
    def forward(self,x):
        '''
        :param x: source word representation vector
        :return: translated word (vector) according to the generator transformation
        '''
        return self.transformation(x)

In [38]:
class Discriminator(Module):
    ''''''
    def __init__(self, dim_in, dim_hid, dim_out):
        '''
        :param dim_in: size of the embbeded space vector
        :param dim_hid: size of the hidden layers
        :param dim_out: output size (being 2 in our problem either source or target)
        '''
        super(Discriminator, self).__init__()
        self.linear1 = Linear(dim_in, dim_hid)
        self.linear2 = Linear(dim_hid, dim_hid)
        self.linear3 = Linear(dim_hid, dim_out)
        #self.activation3 = Softmax()
        
    def forward(self,x):
        '''
        :param x: input word representation (can be eihter transformed source or target)
        :return : a score vector of size dim_out
        '''
        output = LeakyReLU(self.linear1(x))
        output = LeakyReLU(self.linear2(output))
        output = self.linear3(output)
        return output

In [35]:
# Force prtogonality 
def orthogonality(matrix, beta):
    result = (1+beta) * matrix
    result -= beta*(matrix * matrix.T) * matrix 
    return result

## Données

Nous allons utiliser les word embbedings monolingues déjà disponibles sur les git de facebook
- https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md
- https://github.com/facebookresearch/fastText
- https://github.com/facebookresearch/MUSE

In [83]:
filename = 'wiki.fr.vec'
with open(filename) as f:
    lines = f.readlines()
    print(lines)

In [86]:
x = [[1,2,1,5,4,6,5],[1,2,1,5,4,6,5]]
X = np.array(25*x)
y = [[5,4,9,5,2,6,4],[1,2,1,5,4,6,5]]
Y = np.array(25*y)

## Apprentissage

In [87]:
emb_space_size = 300
discriminator_hid_size = 2048
output_size = 2

G = Generator(emb_space_size)
D = Discriminator(emb_space_size, discriminator_hid_size, output_size)

m = LogSoftmax()
L = NLLLoss()

d_optimizer = torch.optim.SGD(D.parameters(), lr=0.1, weight_decay=0.95)
g_optimizer = torch.optim.SGD(G.parameters(), lr=0.1, weight_decay=0.95)

"""========================== ADVERSARIAL TRAINING =================================="""

X = torch.from_numpy(X.astype(float))
Y = torch.from_numpy(Y)

num_epochs = 3
bs = 26 # batch_size
d_nb_batches = int(len(Y) / bs)
g__nb_batches = int(len(X) / bs)

for epoch in range(num_epochs):
            for j in range(nb_batches):
                ''' 1 Train discriminator on source+target data '''
                D.zero_grad()
                
                ''' Constructing mini batches '''
                # 1A Train on target data (real data)
                d_real_data = Variable(torch.FloatTensor(Y[j * bs:(j + 1) * bs, :]))
                d_real_prediction = D(d_real_data)
                d_real_error = L(m(d_real_prediction), Variable(torch.zeros(bs)))
                d_real_error.backward()
                d_optimizer.step()
                
                # 1B Train on source data (fake data)
                d_gen_input = Variable(torch.FloatTensor(X[j * bs:(j + 1) * bs, :]))
                d_fake_data = G(d_gen_input).detach()
                d_fake_prediction = D(d_fake_data)
                d_fake_error = L(m(d_fake_prediction), Variable(torch.ones(bs)))
                d_fake_error.backward()
                d_optimizer.step()
           
                ''' 2 Train generator on D but do not train D here'''
                G.zero_grad()
                
                gen_input = Variable(torch.FloatTensor(X[j * bs:(j + 1) * bs, :]))
                d_fake_data = G(gen_input)
                dg_fake_prediction = D(d_fake_data)
                g_error = L(m(dg_fake_prediction), Variable(torch.ones(bs)))
                g_error.backward()
                g_optimizer.step()
                
                # Orthogonality
                G.parameters = orthogonality(G.parameters, 0.01)
                
    

<class 'torch.DoubleTensor'>

    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    1     2     1     5     4     6     5
    

TypeError: torch.FloatTensor constructor received an invalid combination of arguments - got (torch.LongTensor), but expected one of:
 * no arguments
 * (int ...)
      didn't match because some of the arguments have invalid types: ([31;1mtorch.LongTensor[0m)
 * (torch.FloatTensor viewed_tensor)
      didn't match because some of the arguments have invalid types: ([31;1mtorch.LongTensor[0m)
 * (torch.Size size)
      didn't match because some of the arguments have invalid types: ([31;1mtorch.LongTensor[0m)
 * (torch.FloatStorage data)
      didn't match because some of the arguments have invalid types: ([31;1mtorch.LongTensor[0m)
 * (Sequence data)
      didn't match because some of the arguments have invalid types: ([31;1mtorch.LongTensor[0m)
